In [1]:
import nltk
import json
import pandas as pd

nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Luka\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
model_path = "../../models"

In [3]:
data_path = '../../data/google_translate.jsonl'
with open(data_path, 'r') as f:
    data = [json.loads(line) for line in f]

In [4]:
data[0]

{'message_tree_id': '001199c7-7135-4e64-bee9-48ea862243b4',
 'tree_state': 'ready_for_export',
 'prompt': {'message_id': '001199c7-7135-4e64-bee9-48ea862243b4',
  'user_id': '42398a03-04b3-4028-9d97-1444708e91cd',
  'created_date': '2023-02-04T18:54:01.048716+00:00',
  'text': 'Do you have any information about the Commodore 64?',
  'role': 'prompter',
  'lang': 'en',
  'review_count': 3,
  'review_result': True,
  'deleted': False,
  'synthetic': False,
  'emojis': {'+1': 8, '_skip_reply': 2},
  'replies': [{'message_id': '314ea243-638d-48d6-9f3d-8edf80f85ec2',
    'parent_id': '001199c7-7135-4e64-bee9-48ea862243b4',
    'user_id': '00bbfff8-0e6a-4e46-93cc-65785d78c8bc',
    'created_date': '2023-02-05T00:38:59.927142+00:00',
    'text': "Of course. The Commodore 64 is a 8-bit home computer that was released by Commodore International in August 1982. It is one of the most successful home computers of all time, with over 17 million units sold. It featured 64 kilobytes of RAM, color gra

In [5]:
def tree_to_pairs(tree):
    # get pairs for root prompt and replies
    pairs = message_to_pairs(tree["prompt"])
    return pairs

def message_to_pairs(message):
    pairs = []
    # check who is the input role (we want only pairs where the input is the prompter)
    input_text = message['translation']
    # check if we have any replies to build pairs from
    if 'replies' in message:
        for reply in message['replies']:
            if "translation" not in reply:
                continue
            # get pairs for current input text ang reply
            pairs.append((input_text, reply['translation'], reply['role']))
            # get pairs for reply and its replies
            pairs.extend(message_to_pairs(reply))
    return pairs

In [6]:
pairs = []
for tree in data:
    pairs += tree_to_pairs(tree)
pairs_df = pd.DataFrame(pairs, columns=['prompt', 'reply', 'reply_role'])

mask = pairs_df['reply_role'] == "assistant"
pairs_df = pairs_df[mask]
pairs_df.drop(columns=['reply_role'], inplace=True)
display(pairs_df)

,prompt,reply
0,Ali imate kakšne informacije o Commodore 64?,Seveda. Commodore 64 je 8-bitni domači računal...
1,Ali imate kakšne informacije o Commodore 64?,"Seveda, tukaj je kratek povzetek trenutnega čl..."
7,Ali imate kakšne informacije o Commodore 64?,Commodore 64 (C64) je 8-bitni domači računalni...
9,Ali imate kakšne informacije o Commodore 64?,ja Bi še kaj posebej radi vedeli?
16,Metoda DMAIC v akciji\nProduktna skupina opaža...,"Metoda DMAIC (Define, Measure, Analyze, Improv..."
...,...,...
67035,Katere jezike govorijo v Kostariki?,"Primarni jezik, ki se govori v Kostariki, je š..."
67036,Katere jezike govorijo v Kostariki?,Uradni nacionalni jezik Kostarike je španščina...
67037,Kaj je umetna nevronska mreža?,Umetna nevronska mreža (ANN) je računalniški m...
67038,Kaj je umetna nevronska mreža?,"Umetna nevronska mreža je računalniški model, ..."


### FINE TUNNING

In [7]:
from transformers import AutoModelForSeq2SeqLM, AutoModel, AutoTokenizer

from datasets import *
import numpy as np
import torch

SEED = 42

c:\Users\Luka\miniconda3\envs\NLP\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
metric = load_metric("rouge")

C:\Users\Luka\AppData\Local\Temp\ipykernel_15428\3314070117.py:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [9]:
model_checkpoint = "cjvt/t5-sl-small"
# model_checkpoint = "cjvt/t5-sl-large"

# model_checkpoint = "google/mt5-small"
# model_checkpoint = "google/mt5-base"
# model_checkpoint = "google/mt5-large"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

In [10]:
data = Dataset.from_pandas(pairs_df[["prompt", "reply"]]).train_test_split(test_size=0.3, shuffle=True, seed=SEED)
data

DatasetDict({
    train: Dataset({
        features: ['prompt', 'reply', '__index_level_0__'],
        num_rows: 33259
    })
    test: Dataset({
        features: ['prompt', 'reply', '__index_level_0__'],
        num_rows: 14255
    })
})

In [11]:
def convert_to_features(examples):
    prefix_in = "Uporabnik: "
    examples["prompt"] = [prefix_in + prompt for prompt in examples["prompt"]]
    prefix_out = "Asistent: "
    examples["reply"] = [prefix_out + reply for reply in examples["reply"]]
    
    model_inputs = tokenizer(examples['prompt'], pad_to_max_length=True, max_length=512, truncation=True, return_tensors='pt')

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['reply'], pad_to_max_length=True, max_length=128, truncation=True, return_tensors='pt')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [12]:
train_data, test_data = data["train"], data["test"]
train_data = train_data.map(convert_to_features, batched=True, load_from_cache_file=False)
test_data = test_data.map(convert_to_features, batched=True, load_from_cache_file=False)
train_data

Map:   0%|          | 0/33259 [00:00<?, ? examples/s]c:\Users\Luka\miniconda3\envs\NLP\Lib\site-packages\transformers\tokenization_utils_base.py:2364: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
c:\Users\Luka\miniconda3\envs\NLP\Lib\site-packages\transformers\tokenization_utils_base.py:3606: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.w

Dataset({
    features: ['prompt', 'reply', '__index_level_0__', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 33259
})

In [13]:
train_data[0]

{'prompt': 'Uporabnik: Prosim, napišite mi vrstico pesmi, ki temelji na delih Alligatoaha. Kritizirajte podjetje glede točke, ki jo lahko izberete sami.',
 'reply': 'Asistent: Jaz sem vplivnež, naredil te bom za sledilca. Pokazal ti bom svoj svet, tako je barvit in poln čudes, da ti bom lagal v obraz, a verjameš vsakemu sranju, ki ti ga povem. Ti si moj potrošnik, kupiš vse, kar cenim\n\nKritiziram vlogo vplivnežev v družbi in kako manipulirajo in izkoriščajo svoje sledilce.',
 '__index_level_0__': 57606,
 'input_ids': [23227,
  31388,
  7805,
  31354,
  4791,
  13101,
  182,
  2726,
  288,
  3917,
  31354,
  81,
  2060,
  24,
  6990,
  1921,
  10016,
  52,
  31337,
  285,
  31358,
  17294,
  116,
  17209,
  1321,
  627,
  2815,
  31354,
  81,
  428,
  192,
  30445,
  1534,
  31358,
  1,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
  0,
 

In [14]:
train_data["prompt"]

['Uporabnik: Prosim, napišite mi vrstico pesmi, ki temelji na delih Alligatoaha. Kritizirajte podjetje glede točke, ki jo lahko izberete sami.',
 'Uporabnik: Pojasnite, kako delujejo vodne črpalke.',
 'Uporabnik: Hm, hvala. Imate še kakšne druge predloge ali nasvete?',
 'Uporabnik: Oh čudovito! Ali mi lahko prosim, draga, napišeš, kako lahko uporabim te barve na svoji spletni strani? Na njem imam čudovite rože, vendar ozadje ne bo primerno. Trenutno je roza, vendar ga želim spremeniti v koralno. Oh, ampak ne vem, kako narediti vse to kodiranje bitov in bopov, draga moja.',
 'Uporabnik: Oh, človek, izgubil sem se v mislih in med tipkanjem pozabil spremeniti razporeditev tipkovnice. Ali lahko to popravite, kot to počne Google? Tukaj je predlog:\n"nj ;t njn cfvsq jgty fccbcnfyn? rjnjhsv vj;yj gjkmpjdfnmcz ,tcgkfnyj b yt ,jzcm? xnj rnj-nj pf,th§n ndjb lfyyst',
 'Uporabnik: kako lahko vplivam na to, da ustavim vojno, ne da bi se neposredno vključil v spopade?',
 'Uporabnik: nadrejeni v skri

In [15]:
train_data["reply"]

['Asistent: Jaz sem vplivnež, naredil te bom za sledilca. Pokazal ti bom svoj svet, tako je barvit in poln čudes, da ti bom lagal v obraz, a verjameš vsakemu sranju, ki ti ga povem. Ti si moj potrošnik, kupiš vse, kar cenim\n\nKritiziram vlogo vplivnežev v družbi in kako manipulirajo in izkoriščajo svoje sledilce.',
 'Asistent: Vodne črpalke so naprave, ki lahko premikajo vodo od točke izvora do točke cilja, ne glede na to, ali so te točke na različnih višinah ali razdaljah. Običajno vodna črpalka ne potrebuje zraka za vstop skozi točko izvora, to je tako, ker imajo običajno turbino, ki potiska tok vode (in če bi bil zrak, bi zrak lažje prešel). Vendar pa obstaja veliko različnih vrst vodnih črpalk. Sodobne vodne črpalke uporabljajo motorje, ki vrtijo turbino, ki potiska vodo v smeri izstopne točke. Stare vodne črpalke delujejo ročno (ali živalsko) in temeljijo na uporabi bata, tako da pri premikanju v smeri vstopne točke ne povzročajo upora in zato lahko tekočina teče, čeprav se bat p

In [16]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    
    # Rouge expects a newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]
    
    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    # Extract a few results
    result = {key: value.mid.fmeasure * 100 for key, value in result.items()}
    
    # Add mean generated length
    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    result["gen_len"] = np.mean(prediction_lens)
    
    return {k: round(v, 4) for k, v in result.items()}

In [17]:
from transformers import DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments

# this batch size fits on a GPU with 12GB of RAM with the model_checkpoint = "cjvt/t5-sl-small"
batch_size = 16

name = model_checkpoint.split("/")[-1]
training_args = Seq2SeqTrainingArguments(
    output_dir=f"{model_path}/{name}-finetuned-assistant",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=1,
    predict_with_generate=True,
    fp16=False, # setting this to true gives loss 0.0 at every step for some reason
    push_to_hub=False
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

In [18]:
trainer.train()
trainer.save_model(f"{model_path}/{name}-finetuned-assistant")

c:\Users\Luka\miniconda3\envs\NLP\Lib\site-packages\transformers\optimization.py:407: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
 24%|██▍       | 500/2079 [01:54<06:43,  3.91it/s]

{'loss': 9.1789, 'learning_rate': 1.518999518999519e-05, 'epoch': 0.24}


 48%|████▊     | 1000/2079 [03:49<04:01,  4.47it/s]

{'loss': 3.2992, 'learning_rate': 1.0379990379990381e-05, 'epoch': 0.48}


 72%|███████▏  | 1500/2079 [05:50<02:08,  4.52it/s]

{'loss': 3.2068, 'learning_rate': 5.569985569985571e-06, 'epoch': 0.72}


 96%|█████████▌| 2000/2079 [07:58<00:19,  3.99it/s]

{'loss': 3.1625, 'learning_rate': 7.5998075998076e-07, 'epoch': 0.96}


                                                   
100%|██████████| 2079/2079 [16:19<00:00,  2.12it/s]


{'eval_loss': 2.668219804763794, 'eval_rouge1': 13.9642, 'eval_rouge2': 2.3652, 'eval_rougeL': 12.0961, 'eval_rougeLsum': 13.0059, 'eval_gen_len': 18.7254, 'eval_runtime': 479.875, 'eval_samples_per_second': 29.706, 'eval_steps_per_second': 1.857, 'epoch': 1.0}
{'train_runtime': 979.7341, 'train_samples_per_second': 33.947, 'train_steps_per_second': 2.122, 'train_loss': 4.6529503657646964, 'epoch': 1.0}


In [19]:
input = "Uporabnik: Kdaj je bil prvi pristanek na Luni?" # from train set
input = tokenizer(input, return_tensors="pt").to("cuda")
# output = trainer.model.generate(**input, max_length=128)
outputs = trainer.model.generate(**input, max_length=128, no_repeat_ngram_size=2, num_beams=5, num_return_sequences=5)
# tokenizer.decode(outputs[0], skip_special_tokens=True)
tokenizer.batch_decode(outputs, skip_special_tokens=True)

['Asistent: Ko je bil prvi pristanek na Luni, se je zgodilo, da je bila Luna prva pristanka na Zemlji.',
 'Asistent: Ko je bil prvi pristanek na Luni, se je zgodilo, da je bila Luna prva pristanka na luni.',
 'Asistent: Ko je bil prvi pristanek na Luni, se je zgodilo, da je bila Luna prva pristanka na luni. V tem času je bilo več pristanov, ki so se nahajali v bližini Lune.',
 'Asistent: Ko je bil prvi pristanek na Luni, se je zgodilo, da je bila Luna prva pristanka na luni. V tem času je bilo več pristanov, ki so bili v bližini Lune.',
 'Asistent: Ko je bil prvi pristanek na Luni, se je zgodilo, da je bila Luna prva pristanka na luni. V tem času je bilo več pristanov, ki so se nahajali v bližini Lune, kjer so bili trije pristanki.']

In [20]:
input = "Uporabnik: Kdo je France Prešeren?"
input = tokenizer(input, return_tensors="pt").to("cuda")
# output = trainer.model.generate(**input, max_length=128)
outputs = trainer.model.generate(**input, max_length=128, no_repeat_ngram_size=2, num_beams=5, num_return_sequences=5)
tokenizer.decode(outputs[0], skip_special_tokens=True)
tokenizer.batch_decode(outputs, skip_special_tokens=True)

['Asistent:',
 'Asistent: France Prešeren je pesnik, ki se je rodil v Ljubljani, in je bil rojen leta 1869 v Parizu.',
 'Asistent: France Prešeren je pesnik, ki se je rodil v Ljubljani, in je bil rojen leta 1848 v Parizu.',
 'Asistent: France Prešeren je pesnik, ki se je rodil v Ljubljani leta 1848 in je bil rojen leta 1938 v Parizu.',
 'Asistent: France Prešeren je pesnik, ki se je rodil v Ljubljani leta 1848 in je bil rojen leta 1949 v Mariboru.']

In [22]:
val_results = trainer.evaluate()
test_results = trainer.predict(test_dataset=test_data)

print('Val results: ', val_results)
print('Test results:', test_results.metrics)

1561it [12:12,  2.13it/s]                        
100%|██████████| 891/891 [09:11<00:00,  1.62it/s]

Val results:  {'eval_loss': 2.668219804763794, 'eval_rouge1': 13.9642, 'eval_rouge2': 2.3652, 'eval_rougeL': 12.0961, 'eval_rougeLsum': 13.0059, 'eval_gen_len': 18.7254, 'eval_runtime': 461.0993, 'eval_samples_per_second': 30.915, 'eval_steps_per_second': 1.932, 'epoch': 1.0}
Test results: {'test_loss': 2.668219804763794, 'test_rouge1': 13.9642, 'test_rouge2': 2.3652, 'test_rougeL': 12.0961, 'test_rougeLsum': 13.0059, 'test_gen_len': 18.7254, 'test_runtime': 552.2838, 'test_samples_per_second': 25.811, 'test_steps_per_second': 1.613}
